In [1]:
%%capture
!git clone https://github.com/ronakdm/input-marginalization.git

In [2]:
%%bash
cd input-marginalization
git pull
cd ..

Already up to date.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
save_dir = "/content/gdrive/My Drive/input-marginalization"

Mounted at /content/gdrive


In [4]:
import os,sys,time,math,textwrap

import numpy as np
import torch
import torch.nn as nn

sys.path.append("input-marginalization/generative_model")
import dataset, transformer

root = 'input-marginalization/generative_model/data/wikitext-2'

In [9]:
lr = .00035
context = 150
batch_size = 32
log_interval = 50

heads = 1
depth = 1

torch.manual_seed(0)
device = torch.device("cuda")

In [6]:
train_data = dataset.WikiText2(root, context, dataset.DatasetSplit.train)
valid_data = dataset.WikiText2(root, context, dataset.DatasetSplit.valid)
test_data = dataset.WikiText2(root, context, dataset.DatasetSplit.test)

In [7]:
def evaluate(data):
    model.eval()
    with torch.no_grad():
        loss = 0.
        loader = torch.utils.data.DataLoader(dataset=data,batch_size=batch_size,shuffle=False)
        for i, (x,y) in enumerate(loader):
            x, y = x.permute(1,0).to(device), y.permute(1,0).to(device)
            yhat = model(x).view(-1, train_data.word_count())
            loss += criterion(yhat, y.contiguous().view(-1))

    print()
    model.train()
    return loss / len(loader)

In [8]:
model = transformer.Transformer(context, train_data.word_count(), 400, 40, 900, heads, depth, tied_weights=True).to(device)
count = sum([np.prod(parm.shape) for parm in model.parameters() if parm.requires_grad])
print('Initialized graph with {} parameters'.format(count))

Initialized graph with 15106679 parameters


In [15]:
criterion = nn.NLLLoss()
curr_lr = .0001
clip = .25
best_val_loss = None
epochs = 1
# save = 'model.pt'
save_filename = 'transformer_wikitext2'

train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
print('Initiating training, {} iterations/epoch.'.format(len(train_loader)))

try:
    optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)
    for epoch in range(epochs):
        t0 = time.time()
        val_loss = evaluate(valid_data)
        print('-' * 100)
        print('| checkpoint | epoch {:3d} | time: {:5.2f}s | validation loss {:5.2f} | '
                'validation perplexity {:8.2f}'.format(epoch, (time.time() - t0),
                                                       val_loss, math.exp(val_loss)))
        print('-' * 100)
        print('epoch\t\tms/batch\tlr\tloss\tperplexity')

        if not best_val_loss or val_loss < best_val_loss:
            # with open(save, 'wb') as f:
            #     torch.save(model, f)
            torch.save(model, f"{save_dir}/{save_filename}.pt") # imarg: Save in Drive folder.
            best_val_loss = val_loss

        model.train()
        total_loss = 0.
        t0 = time.time()
        if epoch == 1: optimizer.param_groups[0]['lr'] = curr_lr = lr # finished warmup
        for i, (x,y) in enumerate(train_loader):
            if i % log_interval == 0 and i > 0:
                cur_loss = total_loss / log_interval
                elapsed = time.time() - t0
                print('{:3d} ({:2.1f}%)\t{:5.2f}\t\t{:1.3}\t{:5.2f}\t{:8.2f}'.format(
                    epoch, 100*i/float(len(train_loader)),
                    elapsed * 1000 / log_interval, curr_lr, cur_loss, math.exp(cur_loss)))
                total_loss = 0
                t0 = time.time()

            x, y = x.permute(1,0).to(device), y.permute(1,0).to(device)
            model.zero_grad()
            yhat = model(x).view(-1, train_data.word_count())
            loss = criterion(yhat, y.contiguous().view(-1))
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            total_loss += loss.item()

except KeyboardInterrupt:
    print('Graceful Exit')

Initiating training, 436 iterations/epoch.

----------------------------------------------------------------------------------------------------
| checkpoint | epoch   0 | time:  5.39s | validation loss  5.55 | validation perplexity   258.44
----------------------------------------------------------------------------------------------------
epoch		ms/batch	lr	loss	perplexity
  0 (11.5%)	346.21		0.0001	 5.42	  225.49
  0 (22.9%)	346.86		0.0001	 5.42	  225.12
  0 (34.4%)	345.90		0.0001	 5.39	  219.39
  0 (45.9%)	346.46		0.0001	 5.40	  221.89
  0 (57.3%)	345.90		0.0001	 5.41	  222.86
  0 (68.8%)	345.71		0.0001	 5.37	  214.57
  0 (80.3%)	346.24		0.0001	 5.36	  213.40
  0 (91.7%)	345.40		0.0001	 5.37	  215.19


In [16]:
print('Restoring best checkpointed model...')
# with open(save, 'rb') as f:
#     model = torch.load(f)
model = torch.load(f"{save_dir}/{save_filename}.pt")

test_loss = evaluate(test_data)
print('=' * 89)
print('| end of training | test loss {:5.2f} | test perplexity {:8.2f}'.format(test_loss, math.exp(test_loss)))
print('=' * 89)

Restoring best checkpointed model...

| end of training | test loss  5.47 | test perplexity   238.60


In [17]:
print('\nUncurated samples')
print('-' * 89)

def sample():
    words = []
    model.eval()
    history = torch.randint(train_data.word_count(), (1, 1), dtype=torch.long).cuda()
    for i in range(context):
        output = model(history)
        word_weights = output[-1].squeeze().exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        word_tensor = torch.Tensor([[word_idx]]).long().cuda()
        history = torch.cat([history, word_tensor], 0)

        words.append(train_data.idx2word[word_idx])

    return '\n'.join(textwrap.wrap(' '.join(words),80))

for i in range(5):
    print('({})'.format(i), sample())


Uncurated samples
-----------------------------------------------------------------------------------------
(0) stern , its responsibility for mysterious place during and the fifth <unk>
claimed to without former activated . Future ; gags was expensive while alerted
the release through several hours of Ireland compensation for Eva Perón 's scene
, with word who because parasites of Arabia to <unk> gave its hundred densely
techniques along the Pakistan and had had previously permitted widely divine
responses to 53 . The three government . In 1969 he recognized summer in the
prospect of transport to the <unk> to create a <unk> responsible for the <unk> ,
eventually designed Arne the <unk> . <eos> <eos> = <eos> = = = = <eos> = <eos>
<eos> Following six in the sacraments @-@ awarded for his <unk> reconnaissance
who were owned the title = = = = Ely umpires was arrival residents in 37 April
demo County . She material when the Republic of it was
(1) where this evidence . The song , assumes t